In [1]:
# -*- coding: utf-8 -*-

# 匯入模型
import tkinter as tk
from tkinter.font import Font
from PIL import Image, ImageTk
from tkinter.ttk import*
from tkinter import filedialog
import cv2
import numpy as np
import dlib
from math import hypot
import math

# 讀取人臉辨識模型
detector = dlib.get_frontal_face_detector()

# 讀取人臉辨識之特徵模型
predictor = dlib.shape_predictor("Model/shape_predictor_68_face_landmarks.dat")

## openimg_file
#### 一、打開貼圖圖片來更換GUI影像
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名
2. 使用PIL中的Image來開啟影像，並更改大小為(320,250)
3. 使用ImageTk中的PhotoImage將影像轉為Tk格式
4. 使用config與image將原來的label_img更換影像

#### 二、圖片的格式轉為黑底的貼圖格式
1. 使用cv2中的split來分割三通道(R,G,B)
2. 使用numpy中的empty建立空白的陣列來存放將要處理的影像，並將此陣列存取的元素規定為整數(int)
3. 多巢迴圈(高與寬)與條件式來存放像素：
    a. 背景為白色之外的區域之像素存原來貼圖的像素
    b. 背景為白色的區域之像素存黑色
4. 使用cv2中的imwrite來儲存背景為黑的貼圖

In [2]:
def openimg_file():
    
    # 開啟檔案
    filename = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("png files","*.png"),("jpeg files","*.jpg")])
    
    # 更換影像
    img = Image.open(filename).resize((250,250))
    imgtk = ImageTk.PhotoImage(img)
    label_img.config(image = imgtk)
    label_img.image = imgtk
    
    # 打開影像
    postimg = cv2.imread(filename)
    
    # 分割RGB通道
    r,g,b = cv2.split(postimg)

    # 建立放置背景為黑的貼圖之空陣列，並指定此陣列只存放整數
    black_img = np.empty(postimg.shape)
    black_img = black_img.astype(int)

    # 跑寬與高的多巢迴圈，設立條件式：將背景為白色之外的區域之像素存原來貼圖的像素;反之，背景為白色的區域之像素存黑色
    for h in range(black_img.shape[0]):
        for w in range(black_img.shape[1]):
            if ((b[h][w]) == 255)&((g[h][w]) == 255)&((r[h][w]) == 255):
                black_img[h][w][0]=0
                black_img[h][w][1]=0
                black_img[h][w][2]=0
            else:
                black_img[h][w][0]=postimg[h][w][0]
                black_img[h][w][1]=postimg[h][w][1]
                black_img[h][w][2]=postimg[h][w][2]

    # 儲存背景為黑的貼圖影像
    cv2.imwrite("Image_posting/black_img.jpg", black_img)

## rotate
1. 輸入為影像、角度、中心點及放大尺度
2. 輸出為旋轉影像

In [3]:
def rotate(image, angle, center = None, scale = 1.0):
    
    # 取畫面寬高
    (h, w) = image.shape[:2]
    
    # 若中心點為無時，則中心點取影像的中心點
    if center is None:
        center = (w / 2, h / 2)
    
    # 產生旋轉矩陣Ｍ(第一個參數為旋轉中心，第二個參數旋轉角度，第三個參數：縮放比例)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    
    # 透過旋轉矩陣進行影像旋轉
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

## face_post
1. 輸入為原始影像,貼圖,人臉偵測器, 人臉特徵辨識器, 放大尺度, 中心點放置位置
2. 輸出為貼圖後的影像

In [4]:
def face_post(frame, black_img, detector, predictor, scaling, location):
    
    # 人臉偵測
    face = detector(frame)
    
    # 畫面轉為灰階
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    for face in face:
        
        # 人臉特徵偵測（給予灰階畫面與人臉）
        landmarks = predictor(frame_gray, face)
    
        # 獲得特徵位置
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)
        chin = (landmarks.part(8).x, landmarks.part(8).y)
        
        # 計算旋轉角度
        nose_angle = float(-math.atan((right_nose[1]-left_nose[1])/(right_nose[0]-left_nose[0]))*180/3.14159)
        
        # 貼圖旋轉
        black_img = rotate(black_img, nose_angle)
        
        # 寬高大小(皆以鼻子大小)
        nose_width = int(hypot(left_nose[0]-right_nose[0],left_nose[1]-right_nose[1])*scaling)
        nose_height = int(nose_width)
        
        # 計算中心點(鼻子與頭)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        center_head = (int(center_nose[0])-1.5*(chin[1]-center_nose[1])*math.sin(nose_angle*3.14/180),int(center_nose[1]-1.2*(chin[1]-center_nose[1])))
        
        if location =="Nose":
            center = center_nose
            
        if location =="Head":
            center = center_head
        
        # 計算左上角與右下角
        top_left = (int(center[0]-nose_width/2),int(center[1]-nose_width/2))
        bottom_right = (int(center[0]-nose_width/2),int(center[1]-nose_width/2))
        
        # 用上述的寬高大小來更改貼圖大小
        post_img = cv2.resize(black_img, (nose_width, nose_height))
        
        # 轉為灰階
        post_img_gray = cv2.cvtColor(post_img, cv2.COLOR_BGR2GRAY)
        
        # 二值化
        _, post_mask = cv2.threshold(post_img_gray, 25, 255, cv2.THRESH_BINARY_INV)
        
        if top_left[1]<0 or top_left[0]<0:
            break
            
        area = frame[top_left[1]: top_left[1]+nose_height, top_left[0]: top_left[0]+nose_width]
        area_no = cv2.bitwise_and(area,area,mask=post_mask)
        final = cv2.add(area_no, post_img)
        frame[top_left[1]: top_left[1]+nose_height, top_left[0]: top_left[0]+nose_width] = final

    return frame

## opencv_frame
#### 點下Start的按鈕就立即觸發函數opencv_frame，將分為開啟OpenCV的攝影機、影片或影像：
一、當選擇為Camera，則啟動攝影機：
1. 使用cv2中的VideoCapture來啟動攝影鏡頭
2. 無限迴圈來獲取當前畫面
        a. 使用cv2中的resize來更改影像大小
        b. 使用cv2中的imshow來顯現畫面
        c. 使用cv2中的waitKey來決定畫面速度與跳出無限迴圈的按鍵
3. 使用cv2中的release與destroyAllWindows來銷毀視窗與畫面

二、當選擇為Image，則開啟選擇檔案視窗，來選擇欲開啟的相片：
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名
2. 使用cv2中的imread開啟相片
3. 使用cv2中的namedWindow與imshow來顯現畫面

三、當選擇為Video，則開啟選擇檔案視窗，來選擇欲開啟的影片：
1. 使用filedialog.askopenfilename來跳出檔案窗格選擇，可以調整預顯示的副檔名

In [5]:
def opencv_frame():
    
    # 讀取已經處理背景為黑的貼圖
    black_pig = cv2.imread("Image_posting/black_img.jpg")
    # 讀取使用者選擇的中心點
    center = imglocvar.get()
    # 讀取使用者選擇的尺度
    scale = float(scaling.get())
    
    # 攝像頭
    if openvar.get()=="Camera":
        VIDEO_IN = cv2.VideoCapture(0)
        while True:
            hasFrame, frame = VIDEO_IN.read()
            frame = cv2.resize(frame, None, fx=0.8, fy=0.8)
            #進行影像辨識或影像貼圖
            frame = face_post(frame, black_pig, detector, predictor, scale, center)
            cv2.imshow("Camera", frame)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        VIDEO_IN.release()
        cv2.destroyAllWindows()
    
    # 相片
    if openvar.get()=="Image":
        imgname = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("png files","*.png"),("jpeg files","*.jpg")])
        img = cv2.imread(imgname)
        #進行影像辨識或影像貼圖
        img = face_post(img, black_pig, detector, predictor, scale, center)
        cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
        cv2.imshow('Image', img)
    
    # 影片
    if openvar.get()=="Video":
        videoname = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("mp4 files","*.mp4"),("avi files","*.avi")])
        VIDEO_IN = cv2.VideoCapture(videoname)
        while True:
            hasFrame, frame = VIDEO_IN.read()
            frame = cv2.resize(frame, None, fx=1.0, fy=1.0)
            #進行影像辨識或影像貼圖
            frame = face_post(frame, black_pig, detector, predictor, scale, center)
            cv2.imshow("Video", frame)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        VIDEO_IN.release()
        cv2.destroyAllWindows()

In [6]:
#建立視窗
app = tk.Tk()

#建立視窗標題
app.title("Face - Sticker")

#建立視窗背景
#app.configure(background = 'white')

#建立視窗大小
app.geometry('340x480')

#設立字體
title_label_Font = Font(family="Times", size=18, underline=1)
subtitle_label_Font_1 = Font(family="Times", size=14)

#建立標籤與位置
#建立兩個畫面布局
#布局一
frame_title = tk.Frame(app)
frame_title.grid(column=0, row=0, ipadx=3, pady=3)

#布局二
frame_image = tk.Frame(app)
frame_image.grid(column=0, row=1, ipadx=3, pady=3)

#布局三
frame_input = tk.Frame(app)
frame_input.grid(column=0, row=2, ipadx=3, pady=3)

#布局四
frame_output = tk.Frame(app)
frame_output.grid(column=0, row=3, ipadx=3, pady=3)

#放置標題
label_Title = tk.Label(frame_title, text="Face - Sticker", font=title_label_Font)
label_Title.grid(column=0, row=0, ipadx=100, pady=0)

#放置副標題
label_Subtitle = tk.Label(frame_title, text="※ Please enter the correct image(.png/.jpg).", font=subtitle_label_Font_1)
label_Subtitle.grid(column=0, row=1, ipadx=5, pady=5)

#放置圖片
img_path = "Image_test/pre-inserted picture.png"
img = Image.open(img_path).resize((250,250))
imgtk = ImageTk.PhotoImage(img)
label_img = tk.Label(frame_image, image=imgtk)
label_img.grid(column=0, row=0, ipadx=5, pady=0)

#輸入貼圖的放大倍數
label_scaling = tk.Label(frame_input, text = "Scaling ", font=subtitle_label_Font_1)
label_scaling.grid(column=0, row=0, pady=0)
scaling = tk.Entry(frame_input)
scaling.grid(column=1, row=0, pady=0)

#選擇貼圖放置的位置
label_imgloc = tk.Label(frame_input, text = "Location", font=subtitle_label_Font_1)
label_imgloc.grid(column=0, row=1, pady=0)
imglocvar = tk.StringVar()
imglocCB = Combobox(frame_input, textvariable = imglocvar)
imglocCB["value"]=("Nose","Head","Eye","Mouth")
imglocCB.grid(column=1, row=1, padx=10)

#選擇貼圖為影片,相片或攝影機串流
label_open = tk.Label(frame_input, text = "Source", font=subtitle_label_Font_1)
label_open.grid(column=0, row=2, pady=0)
openvar = tk.StringVar()
openCB = Combobox(frame_input, textvariable = openvar)
openCB["value"]=("Image","Video","Camera")
openCB.grid(column=1, row=2, padx=10)

#放置按鈕與位置
imgButton = tk.Button(frame_output, text = 'Sticker', command = openimg_file)
imgButton.grid(column=0, row=2, pady=10)

openButton = tk.Button(frame_output, text = 'Start', command = opencv_frame)
openButton.grid(column=1, row=2, pady=10)

#程式開始迴圈
app.mainloop()